In [ ]:
from pyspark.sql.functions import pandas_udf, col, regexp_replace, current_timestamp
from pyspark.sql.types import FloatType
import numpy as np
import pandas as pd

In [ ]:
dbutils.widgets.text("schema_name", "")
schema_name = dbutils.widgets.get("schema_name")
catalog_name = 'shirlywang_sandbox'
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")

In [ ]:
coffee = spark.table("dnb_wave_5.sandbox.coffee_raw")
weather = spark.table(f"{catalog_name}.{schema_name}.location_weather_raw")

In [ ]:
display(weather)

In [ ]:
import pyspark.sql.functions as F
# Join coffee and weather on coffee Country of Origin and weather country
joined = coffee.join(weather, 
                     (coffee["`Country of Origin`"] == weather["Country"]))\
                         .drop("country", "ID")

In [ ]:
@pandas_udf(FloatType())
def pandas_array_avg(arrays: pd.Series) -> pd.Series:
    return arrays.apply(lambda arr: float(np.mean(arr)))

array_columns = ["temperature_2m_max", "temperature_2m_min", "daylight_duration", "uv_index_max", "rain_sum", "showers_sum", "snowfall_sum"]  

for col_name in array_columns:
    avg_col_name = f"{col_name}_avg"
    joined = joined.withColumn(avg_col_name, pandas_array_avg(joined[col_name]))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2995129008927707>, line 9
      7 for col_name in array_columns:
      8     avg_col_name = f"{col_name}_avg"
----> 9     joined = joined.withColumn(avg_col_name, pandas_array_avg(joined[col_name]))

NameError: name 'joined' is not defined

In [ ]:
columns_to_select = ["Country of Origin", "Region", "Variety", "Processing Method", "Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", "Uniformity", "Clean Cup", "Sweetness", "Overall", "Defects", "Total Cup Points", "Moisture Percentage", "Category One Defects", "Quakers", "Color", "Category Two Defects", "latitude", "longitude", "elevation", "temperature_2m_max_avg", "temperature_2m_min_avg", "daylight_duration_avg", "uv_index_max_avg", "rain_sum_avg", "showers_sum_avg"]

refined = joined.select(*[col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in columns_to_select])

In [ ]:
refined = refined.withColumn("created_at", current_timestamp())
refined.write \
    .format("delta")\
    .mode("ignore")\
    .saveAsTable(f"{catalog_name}.{schema_name}.coffee_weather_refined")